This notebook presents the tasks of reading, cleaning, and wrangling data for geocoding of current 
Adult Education locations in North Alabama.

The business use case for this project stemmed from 


In [382]:
import pandas as pd
import numpy as np


In [385]:
#conda install -c conda-forge folium

In [383]:
import folium 
from folium.plugins import MarkerCluster

In [193]:
##Read the file and start looking into the existing format.
##The file is an excel file with 3 sheets. 


In [384]:
xls = pd.ExcelFile('C:/Users/User/OneDrive/AdultEd/AdultEdCoordinates1.xlsx')
df1=pd.read_excel(xls,'Northeast')
df2=pd.read_excel(xls, 'WallaceStateHanceville')
df3=pd.read_excel(xls, 'Calhoun')

In [217]:
#df1.head(5)

In [218]:
#df3.head(5)

As evident from the above printout, the 3 different sheets have discrepancies: columns not named/not univormly named, unevent number of columnts, etc. 
Before merging the 3 sheets into one file, the following data cleaning tasks need to be performed:
name columns, move the data from rows to columns (i.e. service types, county name),

In [386]:
#Cleaning out the first tab. There is inforamtion that needs to be moved ot the columns (County, Service).
#this information is included as table subheadings, i.e. row 1, 5, etc.

df1.head(25)

,GED Class Locations and Schedules,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Place,Address,Coordinates,NaN,NaN
1,DeKalb County Class Locations and Schedules,NaN,NaN,NaN,NaN
2,Fort Payne Career Center,"2100 Jordan Road SW Fort Payne, AL 35968 , Sui...","34.4284804, -85.7609591",NaN,NaN
3,Ider High School,"1064 Crabapple Ln, Ider, AL 35981","34.710593, -85.674351",NaN,NaN
4,Northeast AL Community College,"138 AL Hwy 35, Rainsville AL 35771","34.4953643, -85.8492625",NaN,NaN
5,Jackson County Class Locations and Schedules,NaN,NaN,NaN,NaN
6,Northeast AL Community College,"138 AL Hwy 35, Rainsville AL 35771","34.4953643, -85.8492625",NaN,NaN
7,Scottsboro Adult Education Center,"906 South Scott Street, Scottsboro AL 35768","34.6579641, -86.0315051",NaN,NaN
8,Marshall County Class Locations and Schedules,NaN,NaN,NaN,NaN
9,Albertville Career Center,"5920 Hwy 431, Albertville AL 35950","34.2637015,-86.1872064",NaN,NaN


In [387]:
#rename columns
df1.rename(columns={'GED Class Locations and Schedules': 'Location', 'Unnamed: 1': 'Address', 'Unnamed: 2': "Coordinates", 'Unnamed: 3':'County', 'Unnamed: 4': "Service"}, inplace=True)

In [253]:
df1

,Location,Address,Coordinates,County,Service
2,Fort Payne Career Center,"2100 Jordan Road SW Fort Payne, AL 35968 , Sui...","34.4284804, -85.7609591",Dekalb,GED
3,Ider High School,"1064 Crabapple Ln, Ider, AL 35981","34.710593, -85.674351",Dekalb,GED
4,Northeast AL Community College,"138 AL Hwy 35, Rainsville AL 35771","34.4953643, -85.8492625",Dekalb,GED
6,Northeast AL Community College,"138 AL Hwy 35, Rainsville AL 35771","34.4953643, -85.8492625",Jackson,GED
7,Scottsboro Adult Education Center,"906 South Scott Street, Scottsboro AL 35768","34.6579641, -86.0315051",Jackson,GED
9,Albertville Career Center,"5920 Hwy 431, Albertville AL 35950","34.2637015,-86.1872064",Marshall,GED
10,Arab Adult Education Center,"153 South Main Street,Arab AL 35016","34.3148534, -86.4953563",Marshall,GED
11,Guntersville Adult Education Center,"Finley Plaza - 1415 Sunset Dr. Guntersville, ...","34.3525319, -86.3136567",Marshall,GED
12,Snead State Community College,"400 W Mann Ave, Boaz AL 35957","34.1996251, -86.1682008",Marshall,GED
18,Collinsville Public Library,"151 Main Street, Collinsville AL 35961","34.2638688, -85.8604296",Dekalb,ESL


In [388]:
conds = [(df1.index >= 0) & (df1.index <= 4), 
         (df1.index >=6) & (df1.index<=7),
         (df1.index>=9) & (df1.index<=12),
         (df1.index>=18) & (df1.index <=20),
         (df1.index==22),
         (df1.index>=24) & (df1.index<=26),
         (df1.index==32),
         (df1.index==34)
         
        ]

# Set up target values (in the same order as your conditions)
choices = ['Dekalb', 'Jackson', "Marshall", 'Dekalb','Jackson','Marshall', "Dekalb", "Marshall" ]

df1['County'] = np.select(conds, choices)

In [389]:
##Populate the Service column with the type of service, this information was included in rows, as a table subheading in the excel spreadsheet

df1.loc[2:12, 'Service'] = 'GED'
df1.loc[18:26, 'Service'] = "ESL"
df1.loc[31:34, "Service"] = 'Reading'

In [390]:
 
df1=df1.dropna(subset=['Service','Coordinates'])

#dat.dropna(subset=[col_list])
#dat.dropna(how='all')
#df1.head(35)


In [391]:
df1.head(35)

,Location,Address,Coordinates,County,Service
2,Fort Payne Career Center,"2100 Jordan Road SW Fort Payne, AL 35968 , Sui...","34.4284804, -85.7609591",Dekalb,GED
3,Ider High School,"1064 Crabapple Ln, Ider, AL 35981","34.710593, -85.674351",Dekalb,GED
4,Northeast AL Community College,"138 AL Hwy 35, Rainsville AL 35771","34.4953643, -85.8492625",Dekalb,GED
6,Northeast AL Community College,"138 AL Hwy 35, Rainsville AL 35771","34.4953643, -85.8492625",Jackson,GED
7,Scottsboro Adult Education Center,"906 South Scott Street, Scottsboro AL 35768","34.6579641, -86.0315051",Jackson,GED
9,Albertville Career Center,"5920 Hwy 431, Albertville AL 35950","34.2637015,-86.1872064",Marshall,GED
10,Arab Adult Education Center,"153 South Main Street,Arab AL 35016","34.3148534, -86.4953563",Marshall,GED
11,Guntersville Adult Education Center,"Finley Plaza - 1415 Sunset Dr. Guntersville, ...","34.3525319, -86.3136567",Marshall,GED
12,Snead State Community College,"400 W Mann Ave, Boaz AL 35957","34.1996251, -86.1682008",Marshall,GED
18,Collinsville Public Library,"151 Main Street, Collinsville AL 35961","34.2638688, -85.8604296",Dekalb,ESL


Cleaning out the second tab (df2)


In [258]:
df2.head(5)

,Place,Address,Coordinates,County,Service
0,WSCC GED Main Campus,"801 Main St. NW Hanceville, AL 35077","34.0723219, -86.782038",Cullman,GED
1,ESL Main Campus,"801 Main St. NW Hanceville, AL 35077","34.0723219, -86.782038",Cullman,ESL
2,Alabama Career Center,"1201 Katherine St. NW Cullman, AL 35055","34.1811138, -86.850139",Cullman,GED
3,Parkside School,"12431 AL Hwy 69 Cullman, AL 35019","34.2682866, -86.610495",Cullman,GED
4,ESL – The Link of Cullman,"708 9th Street, SE Cullman, AL 35055","34.171699, -86.831159",Cullman,ESL


In [392]:
#Rennaming the columns for consistency across all the datasets
df2.rename(columns={'Place': 'Location'}, inplace=True)

In [394]:
#df2.head(10)
df2.isna()

,Location,Address,Coordinates,County
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,False,False,False,False
6,False,False,True,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [395]:
#df2['Service']='GED'
#adding a Service column and assigning values

service= [(df2.index == 0), 
         (df2.index ==1),
         (df2.index>=2) & (df2.index<=3),
         (df2.index==4), 
         (df2.index==5),
         (df2.index==6),
          (df2.index==7),
          (df2.index==8),
          (df2.index>=9) & (df2.index<=10)
        ]

# Set up target values (in the same order as your conditions)
choices = ['GED', 'ESL', "GED", "ESL", "College Prep/ACT", 'ACE',"GED",'ESL','GED' ]

df2['Service'] = np.select(service, choices)

In [426]:
df2=df2.dropna()

In [427]:
df2.isna()

,Location,Address,Coordinates,County,Service
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,False,False
9,False,False,False,False,False
10,False,False,False,False,False


Moving on to the third tab, where the cleaning tasks are:
1. add a county column and county values
2. re-arrange the column order to match the rest of the data
3. rename the columns to match the rest of the data (e.g. 'Coordinates', not 'GPS')

In [398]:
df3.head(10)

,Unnamed: 0,Location,Address,GPS
0,GED,Athens State,"300 N. Beaty St. Athens, AL","34.80651388885214, -86.96435923863156"
1,GED,Charlie Walker Community Center,"96 Schooner Trail, Toney, AL 35773","34.86312189397718, -86.60963390000002"
2,GED,Community Action Partnership,"3516 Stringfield Rd NW, Huntsville, AL 35810","34.77911438711254, -86.61832817116448"
3,GED,Calhoun Decatur Campus,"6250 US-31, Tanner, AL 35671","34.64814535933342, -86.94932091534214"
4,GED,Decatur Youth Services,"1202 5th Ave SW B, Decatur, AL 35601","34.58999468632191, -86.99437783068423"
5,GED,Good Shepard Church,"3809 Spring Ave. Decatur, AL","34.53576246724109, -87.00194826136843"
6,GED,Hartselle Board of Education Office,"305 College St. NE, Hartselle, AL","34.44764870499331, -86.93126150794733"
7,GED,Huntsville Career Center,2535 Sparkman Drive,"34.76356321935206, -86.59553058465788"
8,GED,Madison County DHR,2206 Oakwood Ave,"34.75060996282883, -86.60239378867142"
9,GED,Calhoun Huntsville Campus,"102 Wynn Drive, Huntsville","34.715902695256155, -86.65568211750839"


In [399]:
##Assigning values to the new County column/df3

conds = [(df3.index == 0), 
         (df3.index >=1) & (df3.index<=2),
        (df3.index==3),
         (df3.index>=4) & (df3.index <=6),
         (df3.index>=7)& (df3.index<=17),
         (df3.index>=18) & (df3.index<=19),
         (df3.index>=20) & (df3.index<=22),
         (df3.index>=23) & (df3.index<=24)
         
        ]

# Set up target values (in the same order as your conditions)
choices = ['Limestone', 'Madison', "Limestone", 'Morgan','Madison','Limestone', "Madison", 'Limestone' ]

df3['County'] = np.select(conds, choices)

In [400]:
df3.head(10)

,Unnamed: 0,Location,Address,GPS,County
0,GED,Athens State,"300 N. Beaty St. Athens, AL","34.80651388885214, -86.96435923863156",Limestone
1,GED,Charlie Walker Community Center,"96 Schooner Trail, Toney, AL 35773","34.86312189397718, -86.60963390000002",Madison
2,GED,Community Action Partnership,"3516 Stringfield Rd NW, Huntsville, AL 35810","34.77911438711254, -86.61832817116448",Madison
3,GED,Calhoun Decatur Campus,"6250 US-31, Tanner, AL 35671","34.64814535933342, -86.94932091534214",Limestone
4,GED,Decatur Youth Services,"1202 5th Ave SW B, Decatur, AL 35601","34.58999468632191, -86.99437783068423",Morgan
5,GED,Good Shepard Church,"3809 Spring Ave. Decatur, AL","34.53576246724109, -87.00194826136843",Morgan
6,GED,Hartselle Board of Education Office,"305 College St. NE, Hartselle, AL","34.44764870499331, -86.93126150794733",Morgan
7,GED,Huntsville Career Center,2535 Sparkman Drive,"34.76356321935206, -86.59553058465788",Madison
8,GED,Madison County DHR,2206 Oakwood Ave,"34.75060996282883, -86.60239378867142",Madison
9,GED,Calhoun Huntsville Campus,"102 Wynn Drive, Huntsville","34.715902695256155, -86.65568211750839",Madison


In [401]:
#renaming the column names for consistency
df3.rename(columns={'Unnamed: 0': 'Service', 'GPS':'Coordinates'}, inplace=True)

#re-arranging the column order for consistency
neworder = ['Location','Address','Coordinates','County', 'Service']
df3=df3.reindex(columns=neworder)


In [428]:
##Merging the 3 dataframes
dta = pd.concat([df1, df2, df3])

In [429]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 2 to 24
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Location     53 non-null     object
 1   Address      53 non-null     object
 2   Coordinates  53 non-null     object
 3   County       53 non-null     object
 4   Service      53 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


The next tasks:
1. prepping the coordinates for mapping
2. using Folium to create a map


In [430]:
dta[['lat', 'lon']] = dta['Coordinates'].str.split(' ', n=1, expand=True)


In [431]:
dta.loc[9, 'lat'] = '34.2637015'
dta.loc[9, 'lon'] = '-86.1872064'
  

In [432]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 2 to 24
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Location     53 non-null     object
 1   Address      53 non-null     object
 2   Coordinates  53 non-null     object
 3   County       53 non-null     object
 4   Service      53 non-null     object
 5   lat          53 non-null     object
 6   lon          53 non-null     object
dtypes: object(7)
memory usage: 4.6+ KB


In [433]:
#dta['lat'] = dta['lon'].replace(',', '', regex=True)
dta['lat'].replace(',','', regex=True, inplace=True)

In [434]:
dta.dtypes

Location       object
Address        object
Coordinates    object
County         object
Service        object
lat            object
lon            object
dtype: object

In [435]:
#pd.to_numeric(dta[['lat']])
dta['lat'] = dta['lat'].apply(pd.to_numeric, errors='coerce')
dta['lon'] =dta['lon'].apply(pd.to_numeric, errors='coerce')

In [442]:

aemap = folium.Map(location=dta[["lat", "lon"]].mean().to_list(), zoom_start=10)

marker_cluster = MarkerCluster().add_to(aemap)
 
for i,r in dta.iterrows():
    location = (r["lat"], r["lon"])
    folium.Marker(location=location,
                      popup = r['Service'],
                      tooltip=r['Service'])\
    .add_to(marker_cluster)
# display the map
aemap

The map shows the pockets of service locations. The grant team can identify the areas without service and focus on researching the needs and possible service locations. 
